In [1]:
import os

In [6]:
model_list = ['baseline', 'gat', 'gnn']
dt_list = ['minus3', 'minus2']
type_data_list = ['noisy', 'normal']
type_out_list = ['v', 'delta']
training_list = ['step', 'rollout']

In [9]:
def create_bash(path_sh, path):
    bash_script_content = f"""#!/bin/bash

    cd {path}
    eval "$(conda shell.bash hook)"
    conda activate myenvPy
    python training_main.py
    """

    with open(path_sh, 'w') as file:
        file.write(bash_script_content)

In [12]:
path_out = '/master/code/send_Alan/sh-f'

In [15]:
# action reaction ...

In [14]:
l = []

for model in model_list:
    for dt in dt_list:
        for type in type_data_list:
            for out in type_out_list:
                for tr in training_list:
                    file_name = f'{model}_type-{type}_dt-{dt}_out-{out}_tr-{tr}.sh'
                    main_path = f'/home/jpierre/v2/new_runs/{model}'

                    create_bash(os.path.join(path_out, file_name), main_path)

['baseline_type-noisy_dt-minus3_out-v_tr-step.sh', 'baseline_type-noisy_dt-minus3_out-v_tr-rollout.sh', 'baseline_type-noisy_dt-minus3_out-delta_tr-step.sh', 'baseline_type-noisy_dt-minus3_out-delta_tr-rollout.sh', 'baseline_type-normal_dt-minus3_out-v_tr-step.sh', 'baseline_type-normal_dt-minus3_out-v_tr-rollout.sh', 'baseline_type-normal_dt-minus3_out-delta_tr-step.sh', 'baseline_type-normal_dt-minus3_out-delta_tr-rollout.sh', 'baseline_type-noisy_dt-minus2_out-v_tr-step.sh', 'baseline_type-noisy_dt-minus2_out-v_tr-rollout.sh', 'baseline_type-noisy_dt-minus2_out-delta_tr-step.sh', 'baseline_type-noisy_dt-minus2_out-delta_tr-rollout.sh', 'baseline_type-normal_dt-minus2_out-v_tr-step.sh', 'baseline_type-normal_dt-minus2_out-v_tr-rollout.sh', 'baseline_type-normal_dt-minus2_out-delta_tr-step.sh', 'baseline_type-normal_dt-minus2_out-delta_tr-rollout.sh', 'gat_type-noisy_dt-minus3_out-v_tr-step.sh', 'gat_type-noisy_dt-minus3_out-v_tr-rollout.sh', 'gat_type-noisy_dt-minus3_out-delta_tr-ste

In [16]:
import torch
import numpy as np

In [20]:
def optimized_getGraph_np(mat_t, radius, threshold=6):
    """
    Optimized function to compute the graph for PyTorch Geometric.

    Args:
    -----
        - `mat_t` (np.array): 2D np array (matrix at a given timestep)
        - `threshold` (float): Distance threshold for connecting vertices

    Returns:
    --------
        - `distList` (torch.Tensor): Tensor of distances and direction cosines.
        - `indices` (torch.Tensor): Tensor of graph indices.
    """
    num_points = mat_t.shape[0]
    # Expand dims to broadcast and compute all pairwise distances
    mat_expanded = np.expand_dims(mat_t, 1)  # Shape: [N, 1, 2]
    all_dists = np.sqrt(np.sum((mat_expanded - mat_t)**2, axis=2))  # Shape: [N, N]

    # Identify pairs below the threshold, excluding diagonal
    ix, iy = np.triu_indices(num_points, k=1)
    valid_pairs = all_dists[ix, iy] < threshold

    # Filter pairs by distance threshold
    filtered_ix, filtered_iy = ix[valid_pairs], iy[valid_pairs]
    distances = all_dists[filtered_ix, filtered_iy]

    # Calculate direction vectors and angles
    direction_vectors = (mat_t[filtered_iy] - mat_t[filtered_ix]) / distances[:, None] 

    radii_i = radius[filtered_ix]
    radii_j = radius[filtered_iy]


    # Normalize distances and create distance vectors

    # Double entries for bidirectional edges
    doubled_indices = np.vstack([np.stack([filtered_ix, filtered_iy], axis=1),
                                 np.stack([filtered_iy, filtered_ix], axis=1)])
    
    doubled_dist_vectors = np.vstack([np.stack([distances, direction_vectors[:, 0], direction_vectors[:, 1], radii_i, radii_j], axis=1),
                                      np.stack([distances, -direction_vectors[:, 0], -direction_vectors[:, 1], radii_j, radii_i], axis=1)])

    # Convert to tensors
    indices_tensor = torch.tensor(doubled_indices.T, dtype=torch.long)
    dist_tensor = torch.tensor(doubled_dist_vectors, dtype=torch.float)

    return dist_tensor, indices_tensor

In [21]:

def optimized_getGraph_th(mat_t, radius, threshold=6):
    """
    Optimized function to compute the graph for PyTorch Geometric.

    Args:
    -----
        - `mat_t` (torch.Tensor): 2D torch Tensor (matrix at a given timestep)
        - `radius` (torch.Tensor): 1D torch Tensor of radii for each point
        - `threshold` (float): Distance threshold for connecting vertices

    Returns:
    --------
        - `distList` (torch.Tensor): Tensor of distances and direction cosines.
        - `indices` (torch.Tensor): Tensor of graph indices.
    """
    num_points = mat_t.shape[0]
    
    # Expand dims to broadcast and compute all pairwise distances
    mat_expanded = mat_t.unsqueeze(1)  # Shape: [N, 1, 2]
    all_dists = torch.sqrt(torch.sum((mat_expanded - mat_t) ** 2, dim=2))  # Shape: [N, N]

    # Identify pairs below the threshold, excluding diagonal
    ix, iy = torch.triu_indices(num_points, num_points, offset=1)
    valid_pairs = all_dists[ix, iy] < threshold

    # Filter pairs by distance threshold
    filtered_ix = ix[valid_pairs]
    filtered_iy = iy[valid_pairs]
    distances = all_dists[filtered_ix, filtered_iy]

    # Calculate direction vectors and angles
    direction_vectors = (mat_t[filtered_iy] - mat_t[filtered_ix]) / distances.unsqueeze(1)

    radii_i = radius[filtered_ix]
    radii_j = radius[filtered_iy]

    # Double entries for bidirectional edges
    doubled_indices = torch.cat([torch.stack([filtered_ix, filtered_iy], dim=1),
                                 torch.stack([filtered_iy, filtered_ix], dim=1)], dim=0)

    doubled_dist_vectors = torch.cat([torch.stack([distances, direction_vectors[:, 0], direction_vectors[:, 1], radii_i, radii_j], dim=1),
                                      torch.stack([distances, -direction_vectors[:, 0], -direction_vectors[:, 1], radii_j, radii_i], dim=1)], dim=0)

    # Convert to tensors
    indices_tensor = doubled_indices.t().long()
    dist_tensor = doubled_dist_vectors.float()

    return dist_tensor, indices_tensor


In [25]:
mat = np.random.uniform(-120, 120, (500, 300, 2))
mat_t = torch.from_numpy(mat)

In [26]:
rads_np = np.ones(300)
rads_th = torch.ones(300)

In [27]:
from tqdm import tqdm

In [54]:
for i in range(mat.shape[0]):
    
    v_np = optimized_getGraph_np(mat[i], rads_np)

    v_th = optimized_getGraph_th(mat_t[i], rads_th)

    if np.any(v_np[1].numpy() != v_th[1].numpy()):
        print('bnvfhdsbv')
    

In [30]:
print(v_np)

(tensor([[ 5.8940e+00, -5.8429e-01, -8.1155e-01,  1.0000e+00,  1.0000e+00],
        [ 2.7736e+00,  9.8916e-01, -1.4684e-01,  1.0000e+00,  1.0000e+00],
        [ 3.9303e+00,  2.5612e-01,  9.6664e-01,  1.0000e+00,  1.0000e+00],
        [ 3.3361e+00, -3.7067e-01,  9.2876e-01,  1.0000e+00,  1.0000e+00],
        [ 1.7184e+00, -9.2219e-01,  3.8673e-01,  1.0000e+00,  1.0000e+00],
        [ 1.8077e+00, -6.5165e-01,  7.5852e-01,  1.0000e+00,  1.0000e+00],
        [ 5.3094e+00,  8.8212e-01,  4.7103e-01,  1.0000e+00,  1.0000e+00],
        [ 5.0067e+00,  6.9472e-01, -7.1928e-01,  1.0000e+00,  1.0000e+00],
        [ 4.6343e+00, -3.1653e-01,  9.4858e-01,  1.0000e+00,  1.0000e+00],
        [ 2.8561e+00,  1.6985e-01,  9.8547e-01,  1.0000e+00,  1.0000e+00],
        [ 4.8552e+00, -9.4787e-01, -3.1865e-01,  1.0000e+00,  1.0000e+00],
        [ 5.8966e+00,  2.4473e-01,  9.6959e-01,  1.0000e+00,  1.0000e+00],
        [ 8.1531e-01,  4.9879e-01,  8.6672e-01,  1.0000e+00,  1.0000e+00],
        [ 4.8301e+00,  2

In [45]:
print(torch.any(v_th[0] != v_np[0]))

tensor(False)


In [57]:
m = v_th[0].numpy() 

print(v_np[1].numpy())

[[  4   6   8   8   8  10  11  11  23  25  25  25  26  26  27  29  30  32
   32  32  33  33  35  35  37  37  39  42  45  46  49  49  51  52  53  54
   55  56  56  58  59  63  66  73  75  80  83  83  85  96  97  98 103 104
  105 105 105 108 109 112 118 119 123 125 128 132 135 137 144 146 148 149
  151 153 153 159 168 173 175 176 176 188 188 201 203 206 210 223 228 229
  233 268 272 282  94 276  76 149 258  50  54 161 120  52 126 203 148 293
  254  82 193 112 290 292 104 240  98 227 192 244 102 202 160 165  63  87
   65 126  60 161 184  91 204 106 143  87 219  88 274 103 147 200 232 253
  266 188 131 240 168 175 205 280 278 292 129 154 221 239 199 145 191 298
  251 187 293 258 164 179 195 213 205 186 205 233 245 206 267 299 219 267
  263 257 236 269 245 286 285 294]
 [ 94 276  76 149 258  50  54 161 120  52 126 203 148 293 254  82 193 112
  290 292 104 240  98 227 192 244 102 202 160 165  63  87  65 126  60 161
  184  91 204 106 143  87 219  88 274 103 147 200 232 253 266 188 131 240
  1

In [56]:
print(v_th[1].numpy())

[[  4   6   8   8   8  10  11  11  23  25  25  25  26  26  27  29  30  32
   32  32  33  33  35  35  37  37  39  42  45  46  49  49  51  52  53  54
   55  56  56  58  59  63  66  73  75  80  83  83  85  96  97  98 103 104
  105 105 105 108 109 112 118 119 123 125 128 132 135 137 144 146 148 149
  151 153 153 159 168 173 175 176 176 188 188 201 203 206 210 223 228 229
  233 268 272 282  94 276  76 149 258  50  54 161 120  52 126 203 148 293
  254  82 193 112 290 292 104 240  98 227 192 244 102 202 160 165  63  87
   65 126  60 161 184  91 204 106 143  87 219  88 274 103 147 200 232 253
  266 188 131 240 168 175 205 280 278 292 129 154 221 239 199 145 191 298
  251 187 293 258 164 179 195 213 205 186 205 233 245 206 267 299 219 267
  263 257 236 269 245 286 285 294]
 [ 94 276  76 149 258  50  54 161 120  52 126 203 148 293 254  82 193 112
  290 292 104 240  98 227 192 244 102 202 160 165  63  87  65 126  60 161
  184  91 204 106 143  87 219  88 274 103 147 200 232 253 266 188 131 240
  1

In [40]:
print(v_np[0].shape)

torch.Size([186, 5])


In [148]:
def getGraph1(mat_t, radius, threshold=6):
    distList = []
    indices = []

    for i in range(mat_t.shape[0]-1):
        # Compute the distance between mat_t[i, :] and all subsequent points
        diff = mat_t[i+1:, :] - mat_t[i, :]
        dist = torch.norm(diff, dim=1)

        print(diff)
        # Find the indices where the distance is below the threshold
        mask = dist < threshold
        valid_j = torch.nonzero(mask)

        print(i)
        print(valid_j)
        print(valid_j.shape)
        print(torch.sum(valid_j))

        if valid_j.shape[0] > 0:
            valid_j = valid_j.squeeze()
            print(f'fnjidnsgi >>> {valid_j}')
            print(mat_t.shape)
            print(valid_j.numel())
            print(i)
            print('-------------')
            print(torch.sqrt(torch.sum(mat_t[i, :] - mat_t[valid_j, :], dim=1)))

        

    return indices, distList


In [337]:
def getGraph1(mat_t, radius, threshold=6, device='cpu'):
    mat_t = mat_t.to(device)
    radius = radius.to(device)
    
    distList = []
    indices = []

    for i in range(mat_t.shape[0]):
        # Compute the distance between mat_t[i, :] and all subsequent points
        diff = mat_t[i+1:, :] - mat_t[i, :].unsqueeze(0)  # Ensure correct broadcasting
        dist = torch.sqrt(torch.sum(diff ** 2, dim = -1))
        #print(dist.shape)

        mask = dist < threshold


        valid_j = torch.nonzero(mask) + i + 1

        if valid_j.shape[0] > 0:

            direction_vectors = diff[mask] / dist[mask].unsqueeze(1)

            i_tensor = torch.full((valid_j.size(0),), i, dtype=torch.long, device=device).view(-1)
            valid_j = valid_j.view(-1)

            indices.append(torch.stack([i_tensor, valid_j], dim=0))
            indices.append(torch.stack([valid_j, i_tensor], dim=0))

            dist_values = dist[mask]
            distList.append(torch.stack([
                dist_values, 
                direction_vectors[:, 0], 
                direction_vectors[:, 1], 
                radius[i_tensor], 
                radius[valid_j]
            ], dim=1))

            distList.append(torch.stack([
                dist_values, 
                -direction_vectors[:, 0], 
                -direction_vectors[:, 1], 
                radius[valid_j], 
                radius[i_tensor]
            ], dim=1))

    if indices:
        indices = torch.cat(indices, dim=-1).to(torch.long)

        # Concatenate all tensors in distList along dimension 0
        distList = torch.cat(distList, dim=0)
    else:
        # Handle the case when no indices are found
        indices = torch.empty(0, 2, dtype=torch.long, device=device)
        distList = torch.empty(0, 5, dtype=torch.float, device=device)

    return indices, distList  

In [330]:
print(mat[15].shape)

(300, 2)


In [331]:
getGraph1(torch.from_numpy(mat[15]), rads_th)

(tensor([[  1, 273,   2,   2,  89, 184,   4,   4,  46,  71,   7, 162,  11,  11,
           11,  80, 249, 270,  16,  16,  16,  16, 112, 131, 281, 287,  21, 160,
           25, 276,  26, 156,  31, 265,  37, 111,  43,  43, 104, 239,  44,  44,
           44, 130, 132, 258,  45, 277,  46,  71,  47, 100,  54,  54,  54,  81,
          114, 157,  58, 208,  65, 255,  67, 287,  74,  93,  77, 178,  78,  78,
          128, 146,  79, 109,  80,  80,  80, 203, 249, 270,  81, 157,  83, 268,
           89, 184,  90, 232,  92, 123,  98, 204, 102, 103, 104, 239, 108, 108,
          108, 137, 217, 235, 110, 214, 112, 281, 117, 117, 117, 117, 142, 192,
          193, 211, 120, 172, 124, 151, 126, 181, 128, 146, 130, 132, 131, 131,
          281, 287, 132, 258, 133, 275, 134, 267, 137, 137, 217, 235, 138, 271,
          142, 142, 193, 211, 144, 197, 147, 252, 149, 275, 152, 206, 154, 242,
          167, 202, 171, 180, 176, 176, 221, 256, 178, 278, 179, 237, 183, 294,
          190, 194, 191, 191, 226, 238, 

In [332]:
from torch_geometric.utils import sort_edge_index

In [341]:
for i in tqdm(range(mat.shape[0])):
    
    v_np = optimized_getGraph_np(mat[i], rads_np)

    v_th = getGraph1(mat_t[i], rads_th)

    v_np = sort_edge_index(v_np[1], v_np[0])
    v_th = sort_edge_index(v_th[0], v_th[1])


    if np.any(v_np[1].numpy() != v_th[1].numpy()):
        if np.max(v_np[1].numpy() - v_th[1].numpy()) > 10**-6:
            print(np.max(v_np[1].numpy() - v_th[1].numpy()))

100%|██████████| 500/500 [00:16<00:00, 30.80it/s]


In [334]:
print(v_np)

(tensor([[  2,   4,   6,   9,  14,  15,  16,  17,  22,  27,  27,  30,  30,  31,
          32,  35,  36,  37,  41,  42,  42,  42,  43,  45,  46,  46,  47,  49,
          49,  49,  51,  57,  58,  60,  60,  60,  62,  62,  64,  66,  70,  73,
          75,  76,  78,  80,  81,  84,  86,  90,  91,  92,  94,  97,  97, 101,
         103, 105, 105, 105, 111, 111, 114, 114, 115, 116, 118, 120, 122, 127,
         127, 132, 134, 136, 137, 138, 139, 139, 140, 141, 141, 145, 146, 151,
         156, 158, 160, 164, 165, 167, 167, 169, 172, 173, 173, 174, 179, 181,
         182, 182, 182, 184, 188, 192, 195, 202, 203, 208, 208, 208, 209, 211,
         211, 213, 213, 213, 214, 215, 217, 221, 222, 222, 228, 230, 231, 232,
         234, 235, 237, 243, 244, 244, 246, 247, 249, 252, 254, 256, 257, 262,
         264, 264, 265, 265, 267, 271, 271, 271, 272, 273, 282, 284, 284, 284,
         285, 286, 286, 286, 287, 290, 291, 291, 293, 294, 294, 294, 295, 296,
         298, 299],
        [291, 164,  31,  57,  4

In [335]:
print(v_th[0])

tensor([[  2,   4,   6,   9,  14,  15,  16,  17,  22,  27,  27,  30,  30,  31,
          32,  35,  36,  37,  41,  42,  42,  42,  43,  45,  46,  46,  47,  49,
          49,  49,  51,  57,  58,  60,  60,  60,  62,  62,  64,  66,  70,  73,
          75,  76,  78,  80,  81,  84,  86,  90,  91,  92,  94,  97,  97, 101,
         103, 105, 105, 105, 111, 111, 114, 114, 115, 116, 118, 120, 122, 127,
         127, 132, 134, 136, 137, 138, 139, 139, 140, 141, 141, 145, 146, 151,
         156, 158, 160, 164, 165, 167, 167, 169, 172, 173, 173, 174, 179, 181,
         182, 182, 182, 184, 188, 192, 195, 202, 203, 208, 208, 208, 209, 211,
         211, 213, 213, 213, 214, 215, 217, 221, 222, 222, 228, 230, 231, 232,
         234, 235, 237, 243, 244, 244, 246, 247, 249, 252, 254, 256, 257, 262,
         264, 264, 265, 265, 267, 271, 271, 271, 272, 273, 282, 284, 284, 284,
         285, 286, 286, 286, 287, 290, 291, 291, 293, 294, 294, 294, 295, 296,
         298, 299],
        [291, 164,  31,  57,  46

In [336]:
np.max(v_np[1].numpy() - v_th[1].numpy())

2.3741345245298362e-07